In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [11]:
torch.version.cuda


In [12]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# print(device)


In [13]:
print(device)


cpu


In [39]:
# ufc_df = pd.read_csv("ufc_combined_0905.csv", index_col=0)
# ufc_df = pd.read_csv("ufc_combined_0911.csv", index_col=0)
ufc_df = pd.read_csv("fight_stats_diff_920.csv", index_col=0)
# ufc_df.drop('Time', axis=1, inplace=True)
ufc_df.head()


,Winner,Ht Diff,Reach Diff,Career W Diff,Career L Diff,W Diff,L Diff,Num Fights Diff,W Perc Diff,Sig Strikes Avg Diff,...,KO Opp Avg Diff,SUB Opp Avg Diff,CTRL Avg Diff,CTRL Opp Avg Diff,Time Avg Diff,Streak Diff,Age Diff,Career Fights Diff,Career W Perc Diff,Stance Diff
0,1,-2,0,-4,-2,1,0,1,100,2.000000,...,0.000000,0.000000,11.000000,0.000000,58.000000,1,-9,-6.0,15.384615,-3
1,0,-5,-4,3,4,1,3,4,20,28.600000,...,0.200000,0.000000,53.000000,-648.400000,-287.200000,-1,3,7.0,-11.904762,0
2,1,2,2,-4,-2,0,1,1,-50,57.500000,...,0.000000,0.000000,42.000000,302.500000,702.000000,0,-1,-6.0,3.809524,3
3,0,2,-3,2,-2,-1,-2,-3,16,29.500000,...,-0.333333,-0.166667,32.166667,239.333333,421.166667,3,1,0.0,12.500000,1
4,1,-2,-3,1,0,1,1,2,0,-31.583333,...,0.000000,0.333333,191.583333,-209.500000,-239.250000,3,0,1.0,1.307190,-1


In [15]:
ufc_df['Date'] = ufc_df['Date'].str[:4]
ufc_df.drop('born_year 1', axis=1, inplace=True)
ufc_df.drop('born_year 2', axis=1, inplace=True)
ufc_df.head()


,Winner,Event,Date,Location,Fighter 1,Fighter 2,Weight class,Stance 1,Career W 1,Career L 1,...,Time Avg 2,Streak 2,Career W 2,Career L 2,Career Fights 2,Career W Perc 2,Ht Diff 2,Reach Diff 2,Age 2,Stance 2
0,1,UFC Fight Night: Blaydes vs. Aspinall,2022,"London, England, United Kingdom",Muhammad Mokaev,Charles Johnson,Flyweight,Orthodox,7,0,...,0.000000,0,11,2,13.0,84.615385,2,0,31,Switch
1,0,UFC Fight Night: Nzechukwu vs. Cutelaba,2022,"Las Vegas, Nevada, USA",Zhalgas Zhumagulov,Charles Johnson,Flyweight,Switch,14,7,...,900.000000,-1,11,3,14.0,78.571429,5,4,31,Switch
2,1,UFC Fight Night: Strickland vs. Imavov,2023,"Las Vegas, Nevada, USA",Charles Johnson,Jimmy Flick,Flyweight,Switch,12,3,...,198.000000,1,16,5,21.0,76.190476,-2,-2,33,Orthodox
3,0,UFC Fight Night: Muniz vs. Allen,2023,"Las Vegas, Nevada, USA",Charles Johnson,Ode Osbourne,Catch Weight,Switch,13,3,...,269.833333,-1,11,5,16.0,68.750000,-2,3,31,Southpaw
4,1,UFC Fight Night: Song vs. Simon,2023,"Las Vegas, Nevada, USA",Cody Durden,Charles Johnson,Flyweight,Southpaw,14,4,...,743.250000,-1,13,4,17.0,76.470588,2,3,32,Switch


In [16]:
ufc_df['Career W Perc 1'] = ufc_df['Career W Perc 1'].round()
ufc_df['Career W Perc 2'] = ufc_df['Career W Perc 2'].round()
ufc_df['W Perc 1'] = ufc_df['W Perc 1'].round()
ufc_df['W Perc 2'] = ufc_df['W Perc 2'].round()


In [17]:
ufc_df.head()


,Winner,Event,Date,Location,Fighter 1,Fighter 2,Weight class,Stance 1,Career W 1,Career L 1,...,Time Avg 2,Streak 2,Career W 2,Career L 2,Career Fights 2,Career W Perc 2,Ht Diff 2,Reach Diff 2,Age 2,Stance 2
0,1,UFC Fight Night: Blaydes vs. Aspinall,2022,"London, England, United Kingdom",Muhammad Mokaev,Charles Johnson,Flyweight,Orthodox,7,0,...,0.000000,0,11,2,13.0,85.0,2,0,31,Switch
1,0,UFC Fight Night: Nzechukwu vs. Cutelaba,2022,"Las Vegas, Nevada, USA",Zhalgas Zhumagulov,Charles Johnson,Flyweight,Switch,14,7,...,900.000000,-1,11,3,14.0,79.0,5,4,31,Switch
2,1,UFC Fight Night: Strickland vs. Imavov,2023,"Las Vegas, Nevada, USA",Charles Johnson,Jimmy Flick,Flyweight,Switch,12,3,...,198.000000,1,16,5,21.0,76.0,-2,-2,33,Orthodox
3,0,UFC Fight Night: Muniz vs. Allen,2023,"Las Vegas, Nevada, USA",Charles Johnson,Ode Osbourne,Catch Weight,Switch,13,3,...,269.833333,-1,11,5,16.0,69.0,-2,3,31,Southpaw
4,1,UFC Fight Night: Song vs. Simon,2023,"Las Vegas, Nevada, USA",Cody Durden,Charles Johnson,Flyweight,Southpaw,14,4,...,743.250000,-1,13,4,17.0,76.0,2,3,32,Switch


In [18]:
##### Eliminate columns


In [19]:
ufc_df.columns


Index(['Winner', 'Event', 'Date', 'Location', 'Fighter 1', 'Fighter 2',
       'Weight class', 'Stance 1', 'Career W 1', 'Career L 1', 'W 1', 'L 1',
       'Num Fights 1', 'W Perc 1', 'Sig Strikes Avg 1', 'Sig Str % 1',
       'Sig Strikes Opp Avg 1', 'Sig Str % Opp 1', 'Strikes Avg 1', 'Str % 1',
       'Strikes Opp Avg 1', 'Str % Opp 1', 'TD Avg 1', 'TD % 1',
       'TD Opp Avg 1', 'TD % Opp 1', 'KD Avg 1', 'KD Opp Avg 1', 'DEC Avg 1',
       'KO Avg 1', 'SUB Avg 1', 'DEC Opp Avg 1', 'KO Opp Avg 1',
       'SUB Opp Avg 1', 'CTRL Avg 1', 'CTRL Opp Avg 1', 'Time Avg 1',
       'Streak 1', 'Ht Diff 1', 'Reach Diff 1', 'Age 1', 'Career Fights 1',
       'Career W Perc 1', 'W 2', 'L 2', 'Num Fights 2', 'W Perc 2',
       'Sig Strikes Avg 2', 'Sig Str % 2', 'Sig Strikes Opp Avg 2',
       'Sig Str % Opp 2', 'Strikes Avg 2', 'Str % 2', 'Strikes Opp Avg 2',
       'Str % Opp 2', 'TD Avg 2', 'TD % 2', 'TD Opp Avg 2', 'TD % Opp 2',
       'KD Avg 2', 'KD Opp Avg 2', 'DEC Avg 2', 'KO Avg 2', 'S

In [121]:
# ufc_df.drop('Location', axis=1, inplace=True)
# ufc_df.drop('TD Opp Avg 1', axis=1, inplace=True)
# ufc_df.drop('TD Opp Avg 2', axis=1, inplace=True)
# ufc_df.drop('CTRL Opp Avg 1', axis=1, inplace=True)
# ufc_df.drop('CTRL Opp Avg 2', axis=1, inplace=True)
# ufc_df.drop('CTRL Avg 1', axis=1, inplace=True)
# ufc_df.drop('CTRL Avg 2', axis=1, inplace=True)
# ufc_df.drop('Stance 1', axis=1, inplace=True)
# ufc_df.drop('Stance 2', axis=1, inplace=True)
# ufc_df.drop('DEC Avg 1', axis=1, inplace=True)
# ufc_df.drop('DEC Avg 2', axis=1, inplace=True)
# ufc_df.drop('SUB Avg 1', axis=1, inplace=True)
# ufc_df.drop('SUB Avg 2', axis=1, inplace=True)
# ufc_df.drop('Career W 1', axis=1, inplace=True)
# ufc_df.drop('Career W 2', axis=1, inplace=True)
# ufc_df.drop('Career L 1', axis=1, inplace=True)
# ufc_df.drop('Career L 2', axis=1, inplace=True)
# ufc_df.drop('Sig Str % Opp 1', axis=1, inplace=True)
# ufc_df.drop('Sig Str % Opp 2', axis=1, inplace=True)
# ufc_df.drop('Time Avg 1', axis=1, inplace=True)
# ufc_df.drop('Time Avg 2', axis=1, inplace=True)
# ufc_df.drop('Event', axis=1, inplace=True)
# ufc_df.drop('Age 1', axis=1, inplace=True)
# ufc_df.drop('Age 2', axis=1, inplace=True)
# # ufc_df.drop('TD % Opp 1', axis=1, inplace=True)
# # ufc_df.drop('TD % Opp 2', axis=1, inplace=True)
# ufc_df.drop('DEC Opp Avg 1', axis=1, inplace=True)
# ufc_df.drop('DEC Opp Avg 2', axis=1, inplace=True)
# ufc_df.drop('SUB Opp Avg 1', axis=1, inplace=True)
# ufc_df.drop('SUB Opp Avg 2', axis=1, inplace=True)
# ufc_df.drop('KD Opp Avg 1', axis=1, inplace=True)
# ufc_df.drop('KD Opp Avg 2', axis=1, inplace=True)
# ufc_df.drop('Career Fights 1', axis=1, inplace=True)
# ufc_df.drop('Career Fights 2', axis=1, inplace=True)
# ufc_df.drop('Num Fights 1', axis=1, inplace=True)
# ufc_df.drop('Num Fights 2', axis=1, inplace=True)
# ufc_df.drop('Career W Perc 1', axis=1, inplace=True)
# ufc_df.drop('Career W Perc 2', axis=1, inplace=True)
# ufc_df.drop('W 1', axis=1, inplace=True)
# ufc_df.drop('W 2', axis=1, inplace=True)
# ufc_df.drop('L 1', axis=1, inplace=True)
# ufc_df.drop('L 2', axis=1, inplace=True)
# ufc_df.head()


,Winner,Date,Fighter 1,Fighter 2,Weight class,W Perc 1,Sig Strikes Avg 1,Sig Str % 1,Sig Strikes Opp Avg 1,Strikes Avg 1,...,Str % Opp 2,TD Avg 2,TD % 2,TD % Opp 2,KD Avg 2,KO Avg 2,KO Opp Avg 2,Streak 2,Ht Diff 2,Reach Diff 2
0,1,2022,Muhammad Mokaev,Charles Johnson,Flyweight,100,2.000000,40.000000,1.0,2.000000,...,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0,2,0
1,0,2022,Zhalgas Zhumagulov,Charles Johnson,Flyweight,20,40.600000,24.023669,48.0,51.200000,...,38.554217,0.00,0.000000,46.153846,0.0,0.000000,0.000000,-1,5,4
2,1,2023,Charles Johnson,Jimmy Flick,Flyweight,50,63.500000,40.317460,65.5,106.000000,...,40.000000,0.00,0.000000,100.000000,0.0,0.000000,0.000000,1,-2,-2
3,0,2023,Charles Johnson,Ode Osbourne,Catch Weight,66,53.666667,26.436782,46.0,85.333333,...,25.806452,0.00,0.000000,22.222222,0.5,0.333333,0.333333,-1,-2,3
4,1,2023,Cody Durden,Charles Johnson,Flyweight,50,23.666667,20.028209,29.0,44.000000,...,25.219298,0.25,14.285714,39.534884,0.0,0.250000,0.000000,-1,2,3


In [397]:
# ufc_df.drop('W 1', axis=1, inplace=True)
# ufc_df.drop('W 2', axis=1, inplace=True)
# ufc_df.drop('L 1', axis=1, inplace=True)
# ufc_df.drop('L 2', axis=1, inplace=True)
ufc_df.head()


,Winner,Event,Date,Location,Fighter 1,Fighter 2,Weight class,Stance 1,Career W 1,Career L 1,...,Time Avg 2,Streak 2,Career W 2,Career L 2,Career Fights 2,Career W Perc 2,Ht Diff 2,Reach Diff 2,Age 2,Stance 2
0,1,UFC Fight Night: Blaydes vs. Aspinall,2022,"London, England, United Kingdom",Muhammad Mokaev,Charles Johnson,Flyweight,Orthodox,7,0,...,0.000000,0,11,2,13.0,85.0,2,0,31,Switch
1,0,UFC Fight Night: Nzechukwu vs. Cutelaba,2022,"Las Vegas, Nevada, USA",Zhalgas Zhumagulov,Charles Johnson,Flyweight,Switch,14,7,...,900.000000,-1,11,3,14.0,79.0,5,4,31,Switch
2,1,UFC Fight Night: Strickland vs. Imavov,2023,"Las Vegas, Nevada, USA",Charles Johnson,Jimmy Flick,Flyweight,Switch,12,3,...,198.000000,1,16,5,21.0,76.0,-2,-2,33,Orthodox
3,0,UFC Fight Night: Muniz vs. Allen,2023,"Las Vegas, Nevada, USA",Charles Johnson,Ode Osbourne,Catch Weight,Switch,13,3,...,269.833333,-1,11,5,16.0,69.0,-2,3,31,Southpaw
4,1,UFC Fight Night: Song vs. Simon,2023,"Las Vegas, Nevada, USA",Cody Durden,Charles Johnson,Flyweight,Southpaw,14,4,...,743.250000,-1,13,4,17.0,76.0,2,3,32,Switch


In [398]:
#Only put in fights after 2010


In [399]:
ufc_df['Date'] = ufc_df['Date'].astype(int)
ufc_df = ufc_df[ufc_df['Date'] >= 2016]


In [400]:
ufc_df.drop('Date', axis=1, inplace=True)
ufc_df.head()


,Winner,Event,Location,Fighter 1,Fighter 2,Weight class,Stance 1,Career W 1,Career L 1,W 1,...,Time Avg 2,Streak 2,Career W 2,Career L 2,Career Fights 2,Career W Perc 2,Ht Diff 2,Reach Diff 2,Age 2,Stance 2
0,1,UFC Fight Night: Blaydes vs. Aspinall,"London, England, United Kingdom",Muhammad Mokaev,Charles Johnson,Flyweight,Orthodox,7,0,1,...,0.000000,0,11,2,13.0,85.0,2,0,31,Switch
1,0,UFC Fight Night: Nzechukwu vs. Cutelaba,"Las Vegas, Nevada, USA",Zhalgas Zhumagulov,Charles Johnson,Flyweight,Switch,14,7,1,...,900.000000,-1,11,3,14.0,79.0,5,4,31,Switch
2,1,UFC Fight Night: Strickland vs. Imavov,"Las Vegas, Nevada, USA",Charles Johnson,Jimmy Flick,Flyweight,Switch,12,3,1,...,198.000000,1,16,5,21.0,76.0,-2,-2,33,Orthodox
3,0,UFC Fight Night: Muniz vs. Allen,"Las Vegas, Nevada, USA",Charles Johnson,Ode Osbourne,Catch Weight,Switch,13,3,2,...,269.833333,-1,11,5,16.0,69.0,-2,3,31,Southpaw
4,1,UFC Fight Night: Song vs. Simon,"Las Vegas, Nevada, USA",Cody Durden,Charles Johnson,Flyweight,Southpaw,14,4,3,...,743.250000,-1,13,4,17.0,76.0,2,3,32,Switch


In [20]:
# categorical features
# cats = ["Fighter 1", "Fighter 2", "Weight class"] #, "Stance 1", "Stance 2", "Location", "Event"]
# cats = ["Fighter 1", "Fighter 2", "Weight class", "Stance 1", "Stance 2", "Location", "Event"]

# for cat in cats:
#     ufc_df[cat].fillna("unk", inplace=True)
#     ufc_df[cat] = ufc_df[cat].astype("category")

cats = ["Fighter 1", "Fighter 2", "Weight class", "Stance 1", "Stance 2", "Location", "Event"]

for cat in cats:
    ufc_df[cat] = ufc_df[cat].fillna("unk").astype("category")



In [21]:
ufc_df[cats] = ufc_df[cats].apply(lambda col: col.cat.codes)
ufc_df.head()


,Winner,Event,Date,Location,Fighter 1,Fighter 2,Weight class,Stance 1,Career W 1,Career L 1,...,Time Avg 2,Streak 2,Career W 2,Career L 2,Career Fights 2,Career W Perc 2,Ht Diff 2,Reach Diff 2,Age 2,Stance 2
0,1,377,2022,83,1486,314,3,1,7,0,...,0.000000,0,11,2,13.0,85.0,2,0,31,4
1,0,553,2022,79,2091,314,3,4,14,7,...,900.000000,-1,11,3,14.0,79.0,5,4,31,4
2,1,613,2023,79,332,959,3,4,12,3,...,198.000000,1,16,5,21.0,76.0,-2,-2,33,1
3,0,543,2023,79,332,1566,1,4,13,3,...,269.833333,-1,11,5,16.0,69.0,-2,3,31,3
4,1,609,2023,79,405,314,3,3,14,4,...,743.250000,-1,13,4,17.0,76.0,2,3,32,4


In [22]:
ufc_df = ufc_df.reset_index(drop=True)


In [23]:
# make sure no missing values
ufc_df.isnull().values.any()


False

In [24]:
missing_columns = ufc_df.isnull().sum()
missing_columns[missing_columns > 0]


Series([], dtype: int64)

In [49]:
# split the dataset
X, y = ufc_df.drop("Winner", axis=1), ufc_df["Winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [50]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(6110, 36) (1528, 36) (6110,) (1528,)


In [51]:
print(y_train.mean(), y_test.mean())


0.5042553191489362 0.49476439790575916


In [52]:
ufc_df.head()

,Winner,Ht Diff,Reach Diff,Career W Diff,Career L Diff,W Diff,L Diff,Num Fights Diff,W Perc Diff,Sig Strikes Avg Diff,...,KO Opp Avg Diff,SUB Opp Avg Diff,CTRL Avg Diff,CTRL Opp Avg Diff,Time Avg Diff,Streak Diff,Age Diff,Career Fights Diff,Career W Perc Diff,Stance Diff
0,1,-2,0,-4,-2,1,0,1,100,2.000000,...,0.000000,0.000000,11.000000,0.000000,58.000000,1,-9,-6.0,15.384615,-3
1,0,-5,-4,3,4,1,3,4,20,28.600000,...,0.200000,0.000000,53.000000,-648.400000,-287.200000,-1,3,7.0,-11.904762,0
2,1,2,2,-4,-2,0,1,1,-50,57.500000,...,0.000000,0.000000,42.000000,302.500000,702.000000,0,-1,-6.0,3.809524,3
3,0,2,-3,2,-2,-1,-2,-3,16,29.500000,...,-0.333333,-0.166667,32.166667,239.333333,421.166667,3,1,0.0,12.500000,1
4,1,-2,-3,1,0,1,1,2,0,-31.583333,...,0.000000,0.333333,191.583333,-209.500000,-239.250000,3,0,1.0,1.307190,-1


In [53]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Copying the data
# X_train_scaled = X_train.copy()
# X_test_scaled = X_test.copy()

# # List of continuous columns
# conts = X.drop(cats, axis=1).columns.values.tolist()

# # Convert the continuous columns to float
# X_train_scaled[conts] = X_train_scaled[conts].astype(float)
# X_test_scaled[conts] = X_test_scaled[conts].astype(float)

# # Scaling the continuous columns
# scaler = StandardScaler()
# X_train_scaled.loc[:, conts] = scaler.fit_transform(X_train[conts])
# X_test_scaled.loc[:, conts] = scaler.transform(X_test[conts])



In [54]:
class UfcDataset(Dataset):
    # def __init__(self, df, cats, conts, targets):
    def __init__(self, df, cats, targets):
        self.X_cats = df[cats].astype(np.int64).values
        # self.X_conts = df[conts].astype(np.float32).values
        self.y = targets.astype(np.float32).values.reshape(-1, 1)

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, i):
        # return [self.X_cats[i], self.X_conts[i], self.y[i]]
        return [self.X_cats[i], self.y[i]]


In [55]:
# train_dataset = UfcDataset(X_train_scaled, cats, conts, y_train)
# test_dataset = UfcDataset(X_test_scaled, cats, conts, y_test)

train_dataset = UfcDataset(X_train_scaled, cats, y_train)
test_dataset = UfcDataset(X_test_scaled, cats, y_test)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [33]:
# Data loaders
bz=32
# bz = 64
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=bz, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=bz, shuffle=False)


In [34]:
# Get the number of categorical and continuous features from the dataset
num_cats = train_dataset.X_cats.shape[1]  # Number of categorical columns
num_conts = train_dataset.X_conts.shape[1]  # Number of continuous columns

# Calculate the total number of features
num_features = num_cats + num_conts

print(f"Number of categorical features: {num_cats}")
print(f"Number of continuous features: {num_conts}")
print(f"Total number of features: {num_features}")


Number of categorical features: 7
Number of continuous features: 71
Total number of features: 78


In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network class
class BinaryClassificationNet(nn.Module):
    def __init__(self, emb_dims, num_conts, fc_layer_sizes, emb_drop, ps):
        super(BinaryClassificationNet, self).__init__()

        # Embedding layers for categorical features
        self.embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in emb_dims])
        self.emb_drop = nn.Dropout(emb_drop)

        # Continuous feature layers
        self.bn_cont = nn.BatchNorm1d(num_conts)

        # Fully connected layers
        all_layer_sizes = [sum([nf for ni, nf in emb_dims]) + num_conts] + fc_layer_sizes
        self.layers = nn.ModuleList()
        for i in range(len(all_layer_sizes)-1):
            self.layers.append(nn.Linear(all_layer_sizes[i], all_layer_sizes[i+1]))
            self.layers.append(nn.ReLU(inplace=True))
            self.layers.append(nn.Dropout(ps[i]))

        # Output layer
        self.output = nn.Linear(all_layer_sizes[-1], 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_cats, x_conts):
        # Assuming x_cats is your categorical input tensor
        # print([x_cat.max().item() for x_cat in x_cats.T])

        # Embedding part
        x_cat_emb = [emb_layer(x_cat) for emb_layer, x_cat in zip(self.embeddings, x_cats.T)]
        x_cat_emb = torch.cat(x_cat_emb, 1)
        x_cat_emb = self.emb_drop(x_cat_emb)

        # Continuous features
        x_conts = self.bn_cont(x_conts)

        # Concatenate embeddings and continuous features
        x = torch.cat([x_cat_emb, x_conts], 1)

        # Fully connected layers
        for layer in self.layers:
            x = layer(x)

        # Output layer
        x = self.output(x)
        return self.sigmoid(x)

# Example usage
input_size = num_features  # For example, if you have 10 features
# model = BinaryClassificationNet(input_size)

# Adjust embedding dimensions to reflect the actual number of categories
# emb_dims = [(1950, 10),  # For the first feature, with max value 1949
#             (1907, 10),  # For the second feature, with max value 1906
#             (15, 6),     # For the third feature, with max value 14
#             (4, 3),      # For the fourth feature, with max value 3
#             (5, 3),      # For the fifth feature, with max value 4
#             (156, 8),    # For the sixth feature, with max value 155
#             (680, 10)]   # For the seventh feature, with max value 679

# num_conts = 71  # Number of continuous features
# fc_layer_sizes = [100, 50]  # Example fully connected layer sizes
# emb_drop = 0.1  # Example dropout rate for embeddings
# ps = [0.5, 0.5]  # Dropout rates for fully connected layers
print(len(cats))

# emb_dims = [(len(ufc_df[cat].unique()), min(50, len(ufc_df[cat].unique())//2)) for cat in cats]
# feature_importance = [1.0]*(len(cats)-2)  + [0.1] + [0.1]# Placeholder for feature importance
# emb_dims = [(len(ufc_df[cat].unique()), min(100, max(5, int(np.log2(len(ufc_df[cat].unique())) * imp))))
#             for cat, imp in zip(cats, feature_importance)]
emb_dims = [(len(ufc_df[cat].unique()), min(50, max(1, int(np.log2(len(ufc_df[cat].unique())))))) for cat in cats]

num_conts = len(conts)  # 71 in your case
fc_layer_sizes = [256, 64, 16]
# fc_layer_sizes = [512, 256, 64] #, 16]  # Adding more capacity and an additional layer

emb_drop = 0.5
ps = [0.5] * 3

# emb_drop = 0.3  # Slightly lower dropout for embedding layers
# ps = [0.6, 0.6, 0.6]  # Progressive dropout rates for fully connected layers

# Initialize the model
model = BinaryClassificationNet(emb_dims, num_conts, fc_layer_sizes, emb_drop, ps)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# Example training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch in train_dl:  # Assuming train_dl is your DataLoader
        x_cats, x_conts, labels = batch  # Unpack the three values from the batch
        optimizer.zero_grad()  # Zero out gradients

        # Forward pass (make sure your model accepts both categorical and continuous inputs)
        outputs = model(x_cats, x_conts)

        # Compute loss and backpropagate
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing/Evaluating the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation during evaluation
    correct = 0
    total = 0
    for batch in test_dl:  # Assuming test_dl is your DataLoader
        x_cats, x_conts, labels = batch  # Unpack categorical, continuous features, and labels
        x_cats, x_conts, labels = x_cats.to(device), x_conts.to(device), labels.to(device)

        # Forward pass
        outputs = model(x_cats, x_conts)

        # Convert outputs to binary predictions (for binary classification)
        preds = (outputs > 0.5).float()  # Threshold at 0.5 for binary classification

        total += labels.size(0)
        correct += (preds == labels).sum().item()  # Count correct predictions

    # Calculate and print accuracy
    print(f"Accuracy: {100 * correct / total:.2f}%")


7
Epoch [1/5], Loss: 0.7313
Epoch [2/5], Loss: 0.7262
Epoch [3/5], Loss: 0.6437
Epoch [4/5], Loss: 0.7529
Epoch [5/5], Loss: 0.6422
Accuracy: 59.75%


In [36]:
# Save the model parameters after testing
model_save_path = 'ufc_model_6014.pth'
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')


Model saved to ufc_model_6014.pth
